In [282]:
!pip install -q langchain_google_genai langchain-core requests

In [283]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
import requests

In [284]:
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

# Step-1: Tool Creation

In [285]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [286]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [287]:
multiply.name

'multiply'

In [288]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [289]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

# Step-2: Tool Binding

In [290]:
llm =ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=gemini_api_key
)

In [291]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--3959bc72-d89f-4162-b7bb-b88365befbef-0', usage_metadata={'input_tokens': 2, 'output_tokens': 36, 'total_tokens': 38, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 26}})

In [292]:
llm_with_tools = llm.bind_tools([multiply])

In [293]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thank you! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--a8bb7b65-0354-461c-a242-ea83a706d2e3-0', usage_metadata={'input_tokens': 58, 'output_tokens': 16, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})

# Step-3: Tool Calling

In [294]:
query = HumanMessage('can you multiply 3 with 1000')

In [295]:
messages = [query]

In [296]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [297]:
result = llm_with_tools.invoke(messages)

In [298]:
messages.append(result)

In [299]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3, "b": 1000}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--d77bdd7d-7b6e-4b4e-b443-5db37d6338fb-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'e9784efe-b50e-42dd-9221-37feda113708', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 97, 'total_tokens': 162, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 76}})]

In [300]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 1000},
  'id': 'e9784efe-b50e-42dd-9221-37feda113708',
  'type': 'tool_call'}]

In [301]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 1000},
 'id': 'e9784efe-b50e-42dd-9221-37feda113708',
 'type': 'tool_call'}

In [302]:
result.tool_calls[0]['args']

{'a': 3, 'b': 1000}

In [303]:
tool_result = multiply.invoke(result.tool_calls[0]['args'])

In [304]:
tool_result

3000

In [305]:
tool_result = multiply.invoke(result.tool_calls[0])

In [306]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='e9784efe-b50e-42dd-9221-37feda113708')

In [307]:
messages.append(tool_result)

In [308]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3, "b": 1000}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--d77bdd7d-7b6e-4b4e-b443-5db37d6338fb-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'e9784efe-b50e-42dd-9221-37feda113708', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 97, 'total_tokens': 162, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 76}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='e9784efe-b50e-42dd-9221-37feda113708')]

# Step-4: Tool Execution

In [309]:
llm_with_tools.invoke(messages)

AIMessage(content='The product of 3 and 1000 is 3000.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--c5125987-11bc-4a65-9d22-c05d98271234-0', usage_metadata={'input_tokens': 102, 'output_tokens': 18, 'total_tokens': 120, 'input_token_details': {'cache_read': 0}})

In [310]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

# Example: Currency Converter

# Tool creation

In [311]:
from google.colab import userdata
currency_api_key = userdata.get('CURRENCY_API')

In [312]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/{currency_api_key}/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [313]:
get_conversion_factor.args

{'base_currency': {'title': 'Base Currency', 'type': 'string'},
 'target_currency': {'title': 'Target Currency', 'type': 'string'}}

In [314]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [315]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'BDT'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1759968002,
 'time_last_update_utc': 'Thu, 09 Oct 2025 00:00:02 +0000',
 'time_next_update_unix': 1760054402,
 'time_next_update_utc': 'Fri, 10 Oct 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'BDT',
 'conversion_rate': 121.7189}

In [316]:
convert.invoke({'base_currency_value':10, 'conversion_rate':121.7189})

1217.189

# Tool Binding

In [317]:
llm =ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=gemini_api_key
)

In [318]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

# Tool Calling

In [319]:
messages = [
    HumanMessage(
        "First, use the get_conversion_factor tool to find the conversion factor between BDT and USD. "
        "Then, use the convert tool to convert 10 BDT to USD using that conversion factor. "
        "Return both the conversion factor and the converted amount."
    )
]

In [320]:
messages

[HumanMessage(content='First, use the get_conversion_factor tool to find the conversion factor between BDT and USD. Then, use the convert tool to convert 10 BDT to USD using that conversion factor. Return both the conversion factor and the converted amount.', additional_kwargs={}, response_metadata={})]

In [321]:
ai_message = llm_with_tools.invoke(messages)

In [322]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "BDT"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--eb8feeee-3a89-4c46-b952-14cd13ad88d5-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'BDT'}, 'id': '494af291-4beb-41a9-b169-7a3a04879e21', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 153, 'total_tokens': 330, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 127}})

In [323]:
messages.append(ai_message)

In [324]:
messages

[HumanMessage(content='First, use the get_conversion_factor tool to find the conversion factor between BDT and USD. Then, use the convert tool to convert 10 BDT to USD using that conversion factor. Return both the conversion factor and the converted amount.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "BDT"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--eb8feeee-3a89-4c46-b952-14cd13ad88d5-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'BDT'}, 'id': '494af291-4beb-41a9-b169-7a3a04879e21', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 153, 'total_tokens': 330, 'input_token_details': {'cache_read': 0}, 'output_token_details':

In [325]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'BDT'},
  'id': '494af291-4beb-41a9-b169-7a3a04879e21',
  'type': 'tool_call'}]

In [326]:
ai_message.tool_calls[0]

{'name': 'get_conversion_factor',
 'args': {'target_currency': 'USD', 'base_currency': 'BDT'},
 'id': '494af291-4beb-41a9-b169-7a3a04879e21',
 'type': 'tool_call'}

In [327]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)



In [328]:
messages

[HumanMessage(content='First, use the get_conversion_factor tool to find the conversion factor between BDT and USD. Then, use the convert tool to convert 10 BDT to USD using that conversion factor. Return both the conversion factor and the converted amount.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "BDT"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--eb8feeee-3a89-4c46-b952-14cd13ad88d5-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'BDT'}, 'id': '494af291-4beb-41a9-b169-7a3a04879e21', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 153, 'total_tokens': 330, 'input_token_details': {'cache_read': 0}, 'output_token_details':

In [329]:
ai_message = llm_with_tools.invoke(messages)

In [330]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--13fe745b-6518-4a13-a66a-f73e2cccfa12-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 10}, 'id': '744eb7d2-871a-49ce-bfda-bae0746cec2d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 405, 'output_tokens': 1067, 'total_tokens': 1472, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1049}})

In [331]:
ai_message.tool_calls

[{'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': '744eb7d2-871a-49ce-bfda-bae0746cec2d',
  'type': 'tool_call'}]

In [332]:
ai_message.tool_calls[0]

{'name': 'convert',
 'args': {'base_currency_value': 10},
 'id': '744eb7d2-871a-49ce-bfda-bae0746cec2d',
 'type': 'tool_call'}

In [333]:
for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'convert':
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    # append this tool message to messages list
    messages.append(tool_message2)



In [334]:
messages

[HumanMessage(content='First, use the get_conversion_factor tool to find the conversion factor between BDT and USD. Then, use the convert tool to convert 10 BDT to USD using that conversion factor. Return both the conversion factor and the converted amount.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "BDT"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--eb8feeee-3a89-4c46-b952-14cd13ad88d5-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'BDT'}, 'id': '494af291-4beb-41a9-b169-7a3a04879e21', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 153, 'total_tokens': 330, 'input_token_details': {'cache_read': 0}, 'output_token_details':

# Tool Execution

In [335]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 10}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--0215d2dd-3bed-4d96-afdb-12f7c50d2eba-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 10}, 'id': '68def5e0-c796-4b4b-a935-387075cd9660', 'type': 'tool_call'}], usage_metadata={'input_tokens': 405, 'output_tokens': 365, 'total_tokens': 770, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 347}})

In [336]:
llm_with_tools.invoke(messages).content

'The conversion factor from BDT to USD is 0.008216.\n10 BDT is equal to 0.08216 USD.'

# Actual Agents

In [339]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [341]:
# --- Step 6: Run the Agent ---
user_query = "convert 10 bdt to usd"
response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: The user wants to convert 10 BDT to USD. I need to first get the conversion factor between BDT and USD, and then use that factor to convert the amount.
Action:
```
{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "BDT",
    "target_currency": "USD"
  }
}
```
Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1759968002, 'time_last_update_utc': 'Thu, 09 Oct 2025 00:00:02 +0000', 'time_next_update_unix': 1760054402, 'time_next_update_utc': 'Fri, 10 Oct 2025 00:00:02 +0000', 'base_code': 'BDT', 'target_code': 'USD', 'conversion_rate': 0.008216}
Thought:Action:
```
{
  "action": "convert",
  "action_input": {
    "base_currency_value": 10,
    "conversion_rate": 0.008216
  }
}
```
Observation: 0.08216
Thought:The previous tool execution successfully converted 10 BDT to 0.08216 USD. I can 